In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
from mpei.scrape_data import get_all_target_links, get_info_from_dir_page
target_links = get_all_target_links()

In [3]:
print(len(target_links))

36


In [4]:
from src.classes import ListUniversityDirection, StudentsPrepWorkflow
sw = StudentsPrepWorkflow()

In [5]:
lud = ListUniversityDirection()
students_dfs = {}
for idx, link in enumerate(target_links):
    url = link.get('href')
    key, ud, students_df = get_info_from_dir_page(url)
    lud[key]=ud
    students_dfs[key] = students_df
    print(f'{(idx+1)}/{len(target_links)}. Загружено направление {ud.properties["name"]}. Мест {ud.places}. Подано заявлений {len(students_df)}.')

1/36. Загружено направление Прикладная математика и информатика МПОВМКС ИВТИ бюджет (Очная) . Мест 50. Подано заявлений 2405.
2/36. Загружено направление Прикладная математика и информатика ММ ИВТИ бюджет (Очная) . Мест 37. Подано заявлений 1171.
3/36. Загружено направление Строительство ИГВИЭ бюджет (Очная) . Мест 25. Подано заявлений 1395.
4/36. Загружено направление Информатика и вычислительная техника ИВТИ бюджет (Очная) . Мест 158. Подано заявлений 4231.
5/36. Загружено направление Прикладная информатика ИнЭИ бюджет (Очная) . Мест 24. Подано заявлений 3094.
6/36. Загружено направление Информационная безопасность ИнЭИ бюджет (Очная) . Мест 33. Подано заявлений 3644.
7/36. Загружено направление Беспроводные технологии ИРЭ бюджет (Очная) . Мест 82. Подано заявлений 1754.
8/36. Загружено направление Электроника и наноэлектроника ИРЭ, ИЭТЭ бюджет (Очная) . Мест 191. Подано заявлений 3003.
9/36. Загружено направление Интеллектуальные системы ИРЭ бюджет (Очная) . Мест 66. Подано заявлени

In [6]:
all_students = []
for key, df in students_dfs.items():
    df = df[df['Примечание'].apply(lambda x: False if 'Зачисляется в другой КГ' in str(x) else True)]
    st_df = df[['СНИЛС или Рег.номер', 'Сумма', 'Оригинал', 'Приоритет']]
    st_df.columns = ['snils', 'points', 'podl', 'priority']
    st_df['snils'] = st_df['snils'].apply(lambda x: x.replace('СНИЛС: ', ''))
    st_df['dir_key'] = key
    st_df['podl'] = st_df['podl'].apply(lambda x: True if 'да' in x else False)
    sw = StudentsPrepWorkflow()
    students = sw.df2StudentWithPriority(st_df)
    all_students += students

In [7]:
sw.list_dataclass2df(all_students)

,snils,points,podl,priority,dir_key
0,16552783796,303,False,3,581bacc
1,16656357100,299,False,1,581bacc
2,16492380589,299,False,1,581bacc
3,16994776356,298,False,2,581bacc
4,19665702418,295,True,1,581bacc
...,...,...,...,...,...
44441,Рег.номер: 4100022,128,True,1,143bacc
44442,Рег.номер: 4100003,128,True,1,143bacc
44443,Рег.номер: 4100021,124,True,1,143bacc
44444,Рег.номер: 4100038,124,True,1,143bacc


In [8]:
list(lud.items())[0:2]

[('581bacc',
  UniversityDirection(key='581bacc', places=50, properties={'name': 'Прикладная математика и информатика МПОВМКС ИВТИ бюджет (Очная) ', 'url': '581bacc.html'}, students=[])),
 ('16bacc',
  UniversityDirection(key='16bacc', places=37, properties={'name': 'Прикладная математика и информатика ММ ИВТИ бюджет (Очная) ', 'url': '16bacc.html'}, students=[]))]

In [9]:
from copy import deepcopy
lud_backup = deepcopy(lud)
all_students_backup = deepcopy(all_students)
sw = StudentsPrepWorkflow()

In [10]:
students = sw.to_StudentsUniqueList(all_students)

In [11]:
students_filtered = sw.filter_StudentsUniqueList(students)

In [12]:
for st in students_filtered:
    st.dirs_priority = {x['priority']:x['dir_key'] for x in st.dirs_priority}
students_filtered

[StudentWithPriorities(snils='19665702418', points=295, podl=True, dirs_priority={1: '581bacc'}),
 StudentWithPriorities(snils='15145007018', points=292, podl=True, dirs_priority={1: '22bacc'}),
 StudentWithPriorities(snils='18990138825', points=288, podl=True, dirs_priority={1: '22bacc'}),
 StudentWithPriorities(snils='17808572812', points=287, podl=True, dirs_priority={1: '581bacc'}),
 StudentWithPriorities(snils='18373598921', points=287, podl=True, dirs_priority={1: '16bacc'}),
 StudentWithPriorities(snils='16403814244', points=286, podl=True, dirs_priority={1: '581bacc'}),
 StudentWithPriorities(snils='18101095223', points=285, podl=True, dirs_priority={1: '975bacc'}),
 StudentWithPriorities(snils='16358648503', points=282, podl=True, dirs_priority={1: '35bacc'}),
 StudentWithPriorities(snils='19098440405', points=282, podl=True, dirs_priority={1: '22bacc'}),
 StudentWithPriorities(snils='19123082446', points=282, podl=True, dirs_priority={1: '4bacc'}),
 StudentWithPriorities(snil

In [13]:
sw.list_dataclass2df(students_filtered)

,snils,points,podl,dirs_priority
0,19665702418,295,True,{1: '581bacc'}
1,15145007018,292,True,{1: '22bacc'}
2,18990138825,288,True,{1: '22bacc'}
3,17808572812,287,True,{1: '581bacc'}
4,18373598921,287,True,{1: '16bacc'}
...,...,...,...,...
1984,17976190433,126,True,{1: '1bacc'}
1985,Рег.номер: 4100021,124,True,{1: '143bacc'}
1986,Рег.номер: 4100038,124,True,{1: '143bacc'}
1987,20164367933,123,True,{1: '5bacc'}


In [14]:
lud.fill_guys(students_filtered)

In [22]:
sw.list_dataclass2df(lud['13bacc'].students)

,snils,points,podl,dirs_priority
0,Рег.номер: 1100782,273,True,{1: '13bacc'}
1,15791105272,271,True,{1: '13bacc'}
2,18529508302,271,True,{1: '13bacc'}
3,20652485150,266,True,{1: '13bacc'}
4,17159817701,263,True,{1: '13bacc'}
...,...,...,...,...
162,19138503778,151,True,{1: '13bacc'}
163,15007958454,149,True,{1: '13bacc'}
164,15983299329,138,True,"{2: '13bacc', 3: '518bacc', 4: '7bacc', 5: '97..."
165,19955286546,137,True,{1: '13bacc'}


In [21]:
lud['10bacc'].places

191

In [57]:
import pandas as pd
df = pd.DataFrame((lud.values()))
del df['students']
df['min'] = df['key'].apply(lambda x: lud[x].get_min_points())
df['name'] = df['properties'].apply(lambda x: x['name'])
del df['properties']
df

,key,places,min,name
0,581bacc,50,243,Прикладная математика и информатика МПОВМКС ИВ...
1,16bacc,37,206,Прикладная математика и информатика ММ ИВТИ бю...
2,533bacc,25,199,Строительство ИГВИЭ бюджет (Очная)
3,14bacc,158,212,Информатика и вычислительная техника ИВТИ бюдж...
4,35bacc,24,247,Прикладная информатика ИнЭИ бюджет (Очная)
5,22bacc,33,249,Информационная безопасность ИнЭИ бюджет (Очная)
6,19bacc,82,149,Беспроводные технологии ИРЭ бюджет (Очная)
7,10bacc,191,145,"Электроника и наноэлектроника ИРЭ, ИЭТЭ бюджет..."
8,20bacc,66,137,Интеллектуальные системы ИРЭ бюджет (Очная)
9,15bacc,103,139,"Биотехнические системы и технологии, приборост..."
